In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import datetime
import os

import tensorflow as tf
import numpy as np

In [0]:
## note: code is largely based on: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/magic_wand/train/train.py

In [0]:
####################  TODO: DATA LOADING  ####################


In [0]:
gest = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z', \
        'period', 'comma', 'exclamation_point', 'apostrophe', 'space', 'backspace', 'done', 'slash', 'quotes', 'unknown']
NUM_GESTURES = len(gest)

####################  BUILD MODEL  ####################
def build_cnn(seq_length):
  """Builds a convolutional neural network in Keras."""
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(
          8, (4, 3),
          padding="same",
          activation="relu",
          input_shape=(seq_length, 3, 1)),  # output_shape=(batch, 128, 3, 8)
      tf.keras.layers.MaxPool2D((3, 3)),  # (batch, 42, 1, 8)
      tf.keras.layers.Dropout(0.1),  # (batch, 42, 1, 8)
      tf.keras.layers.Conv2D(16, (4, 1), padding="same",
                             activation="relu"),  # (batch, 42, 1, 16)
      tf.keras.layers.MaxPool2D((3, 1), padding="same"),  # (batch, 14, 1, 16)
      tf.keras.layers.Dropout(0.1),  # (batch, 14, 1, 16)
      tf.keras.layers.Flatten(),  # (batch, 224)
      tf.keras.layers.Dense(16, activation="relu"),  # (batch, 16)
      tf.keras.layers.Dropout(0.1),  # (batch, 16)
      tf.keras.layers.Dense(NUM_GESTURES, activation="softmax")  # (batch, NUM_GESTURES)
  ])

  model_path = os.path.join("./netmodels", "CNN")
  print("Built CNN.")
  # if not os.path.exists(model_path):
  #   os.makedirs(model_path)
  # model.load_weights("./netmodels/CNN/weights.h5")
  return model, model_path

In [20]:
### Test build CNN function 
SEQ_LENGTH = 300

def test_build_net():
    cnn, cnn_path = build_cnn(SEQ_LENGTH)
    cnn_data = np.random.rand(60, SEQ_LENGTH, 3, 1)
    cnn_prob = cnn(tf.constant(cnn_data, dtype="float32")).numpy()
    assert cnn_prob.shape == (60, NUM_GESTURES)
    assert cnn_path == "./netmodels/CNN"
    assert isinstance(cnn, tf.keras.Sequential)

test_build_net()

Built CNN.


In [0]:
####################  UTIL FUNCTIONS ####################

def reshape_function(data, label):
  reshaped_data = tf.reshape(data, [-1, 3, 1])
  return reshaped_data, label


def calculate_model_size(model):
  print(model.summary())
  var_sizes = [
      np.product(list(map(int, v.shape))) * v.dtype.size
      for v in model.trainable_variables
  ]
  print("Model size:", sum(var_sizes) / 1024, "KB")

In [0]:
####################  MODEL TRAINING ####################

def train_net(model,
model_path,  # pylint: disable=unused-argument
    train_len,  # pylint: disable=unused-argument
    train_data,
    valid_len,
    valid_data,  # pylint: disable=unused-argument
    test_len,
    test_data,
    kind):
  """Trains the model."""
  calculate_model_size(model)
  epochs = 50
  batch_size = 64
  model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",  metrics=["accuracy"])

  ############# Prepare data #############
  # TODO: do we need this reshape
  train_data = train_data.map(reshape_function)
  test_data = test_data.map(reshape_function)
  valid_data = valid_data.map(reshape_function)

  test_labels = np.zeros(test_len)
  idx = 0
  for data, label in test_data:  # pylint: disable=unused-variable
    test_labels[idx] = label.numpy()
    idx += 1
  train_data = train_data.batch(batch_size).repeat()
  valid_data = valid_data.batch(batch_size)
  test_data = test_data.batch(batch_size)

  ############## Train model! ##############
  model.fit(train_data, epochs=epochs, validation_data=valid_data, steps_per_epoch=1000, \
            validation_steps=int((valid_len - 1) / batch_size + 1), callbacks=[tensorboard_callback])
  loss, acc = model.evaluate(test_data)
  pred = np.argmax(model.predict(test_data), axis=1)
  confusion = tf.math.confusion_matrix(
      labels=tf.constant(test_labels),
      predictions=tf.constant(pred),
      num_classes=NUM_GESTURES)
  print(confusion)
  print("Loss {}, Accuracy {}".format(loss, acc))

  ############## Convert model to TFLite! ##############
  # Convert the model to the TensorFlow Lite format without quantization
  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  tflite_model = converter.convert()

  # Save the model to disk
  open("model.tflite", "wb").write(tflite_model)

  # Convert the model to the TensorFlow Lite format with quantization
  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
  tflite_model = converter.convert()

  # Save the model to disk
  open("model_quantized.tflite", "wb").write(tflite_model)

  basic_model_size = os.path.getsize("model.tflite")
  print("Basic model is %d bytes" % basic_model_size)
  quantized_model_size = os.path.getsize("model_quantized.tflite")
  print("Quantized model is %d bytes" % quantized_model_size)
  difference = basic_model_size - quantized_model_size
  print("Difference is %d bytes" % difference)


In [0]:
logdir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

######################## RUN TRAINING ########################
if __name__ == "__main__":
  parser = argparse.ArgumentParser()
  parser.add_argument("--model", "-m")
  parser.add_argument("--person", "-p")
  args = parser.parse_args()

  seq_length = 128

  print("Start to load data...")
  if args.person == "true":
    train_len, train_data, valid_len, valid_data, test_len, test_data = \
        load_data("./person_split/train", "./person_split/valid",
                  "./person_split/test", seq_length)
  else:
    train_len, train_data, valid_len, valid_data, test_len, test_data = \
        load_data("./data/train", "./data/valid", "./data/test", seq_length)

  print("Start to build net...")
  model, model_path = build_cnn(seq_length)

  print("Start training...")
  train_net(model, model_path, train_len, train_data, valid_len, valid_data,
            test_len, test_data, args.model)

  print("Training finished!")